In [1]:
import sys
sys.path.insert(0, '../')

from interface import get_query_emb_batch, load_model, cosine_similarity_batch

from typing import Tuple, List, Dict, Union, Any

from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch

# pth_models = "/home/sondors/Documents/price/BERT_data/data/10-04-2024_Timofey/2801_models_Apple.csv"
pth_models = "/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_models_Apple.csv"
df_models = pd.read_csv(pth_models, sep=';')

pth_offers = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_Apple_triplets_offer_model_train.csv'
df_offers = pd.read_csv(pth_offers, sep=';')

pth_offers_test = '/home/sondors/Documents/price/BERT_data/data/17-04-2024_Timofey/2801_offers_test_Apple.csv'
df_offers_test = pd.read_csv(pth_offers, sep=';')

id_category = {
    # 3902: 'диктофоны, портативные рекордеры',
    # 510402: 'электронные книги',
    # 4302: 'автомобильные телевизоры, мониторы',
    # 2815: 'смарт-часы и браслеты',
    # 3901: 'портативные медиаплееры',
    # 3904: 'портативная акустика',
    2801: 'мобильные телефоны',
    # 3908: 'VR-гарнитуры (VR-очки, шлемы, очки виртуальной реальности, FPV очки для квадрокоптеров)',
    # 510401: 'планшетные компьютеры и мини-планшеты',
    # 2102: 'наушники, гарнитуры, наушники c микрофоном',
    # 3903: 'радиоприемники, радиобудильники, радиочасы',
    # 3907: 'магнитолы',
    # 280801: 'GPS-навигаторы'
    }

df_models = df_models[df_models['category_id'].isin(id_category.keys())].reset_index(drop=True)
df_offers = df_offers[df_offers['category_id'].isin(id_category.keys())].reset_index(drop=True)

df_offers

,name,model_id,offer_id,modelized_by_barcode,category_name,category_id,price,attrs,description,external_category,external_brand,external_type,true_match,false_match
0,Смартфоны Смартфон Apple iPhone 12 A2403 128ГБ...,7883241,1530924893,True,мобильные телефоны,2801,53060.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","Смартфон Apple A2403 iPhone 12 черный, процесс...",Смартфоны,APPLE,Смартфоны,"Apple iPhone 12 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 128Gb, eSIM, Green"
1,Apple iPhone 14 Pro 128ГБ Silver (Серебристый)...,7883526,1626555801,True,мобильные телефоны,2801,118050.0,"[{'id': 14519, 'type': 'b', 'value': 1.0}, {'i...",Apple iPhone 14 Pro — супербыстрый смартфон с ...,Телефоны :: Телефоны Apple iPhone,Apple,NaN,"Apple iPhone 14 Pro 128Gb, Dual nano-SIM, Silver","Apple iPhone 14 Pro 128Gb, Dual nano-SIM, Gold"
2,Смартфон Apple iPhone 12 256Gb Blue (А2403),7883255,1690046919,False,мобильные телефоны,2801,77499.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Смартфон Apple iPhone 12 256Gb Blue (А2403),Смартфоны и телефоны :: Apple :: iPhone 12,NaN,NaN,"Apple iPhone 12 256Gb, nano-Sim+eSIM, Синий","Apple iPhone 12 256Gb, nano-Sim+eSIM, Зеленый"
3,Смартфон Apple iPhone 12 128GB MGJA3HN/A Black,7883241,1540988497,True,мобильные телефоны,2801,62990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...","iPhone 12. Во-первых, это быстро. iPhone 12 по...",Телефоны :: Смартфоны,Apple,Смартфон,"Apple iPhone 12 128Gb, nano-Sim+eSIM, Black","Apple iPhone 12 128Gb, eSIM, Green"
4,"Смартфон Apple iPhone 14 Pro 1 ТБ, Dual nano S...",7883536,1837804366,True,мобильные телефоны,2801,151170.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Новый волшебный способ взаимодействия с iPhone...,Все товары :: Электроника :: Телефоны :: Мобил...,Apple,Смартфон,"Apple iPhone 14 Pro 1Tb, Dual nano-SIM, Deep P...","Apple iPhone 14 Pro 1Tb, Dual eSIM, Space Black"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42715,Смартфон Apple iPhone 15 Plus 128GB eSIM Black...,7883661,1810174582,False,мобильные телефоны,2801,80490.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 15 Plus :: iPhone 15 Plus 1...,Apple,Смартфон,"Apple iPhone 15 Plus 128Gb, Dual eSIM, Black","Apple iPhone 15 512Gb, nano-Sim+eSIM, Черный"
42716,Смартфон Apple iPhone 14 Plus 128Gb Purple (фи...,7883483,1844276843,False,мобильные телефоны,2801,76090.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 1...,Apple,Смартфон,"Apple iPhone 14 Plus 128Gb, nano-Sim+eSIM, Фио...","Apple iPhone 14 Plus 128Gb, nano-Sim+eSIM, Тём..."
42717,Смартфон Apple iPhone 14 Plus 256Gb Purple (фи...,7883496,1844276848,False,мобильные телефоны,2801,88990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 2...,Apple,Смартфон,"Apple iPhone 14 Plus 256Gb, nano-Sim+eSIM, Purple","Apple iPhone 14 Plus 256Gb, Dual nano-SIM, Черный"
42718,Смартфон Apple iPhone 14 Plus 512Gb Blue (голу...,7883515,1851901283,False,мобильные телефоны,2801,93990.0,"[{'id': 3, 'type': 'e', 'value': 676.0}, {'id'...",Возможна доставка в день заказа. Квалифицирова...,Телефоны :: iPhone 14 Plus :: iPhone 14 Plus 5...,Apple,Смартфон,"Apple iPhone 14 Plus 512Gb, nano-Sim+eSIM, Blue","Apple iPhone 14 Plus 512Gb, nano-Sim+eSIM, (PR..."


In [2]:
# ckpt_pth = "/home/sondors/Documents/ColBERT_weights/triples_X1_13_categories_use_ib_negatives/none/2024-01/26/10.49.44/checkpoints/colbert-5387-finish"
# experiment = "colbert-5387"

ckpt_pth = "/home/sondors/Documents/ColBERT_weights/2801_lr04_bsize_210_apple/none/2024-04/18/09.16.10/checkpoints/colbert-187-finish"
experiment = "colbert-187-finish"

doc_maxlen = 300
nbits = 2   # bits определяет количество битов у каждого измерения в семантическом пространстве во время индексации
nranks = 1  # nranks определяет количество GPU для использования, если они доступны
kmeans_niters = 4 # kmeans_niters указывает количество итераций k-means кластеризации; 4 — хороший и быстрый вариант по умолчанию. 

device = "cuda"


In [3]:
checkpoint = load_model(ckpt_pth, doc_maxlen, nbits, kmeans_niters, device)

In [4]:
from typing import Tuple, List, Dict, Union, Any
from colbert.modeling.checkpoint import Checkpoint

import numpy as np
import torch

import xgboost as xgb
import numpy as np

from sklearn.metrics import precision_score, recall_score, accuracy_score

def get_query_emb(sentences: List[str], checkpoint: Checkpoint, batch_size: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.

    Returns:
        np.ndarray: An array of embeddings for the input sentences.
    """
    return checkpoint.queryFromText(sentences, bsize=batch_size).to("cpu").numpy()

def get_query_emb_batch(sentences: List[str], checkpoint: Checkpoint, batch_size: int, batch_size2: int) -> np.ndarray:
    """
    Generate embeddings for a list of sentences in batches using the provided checkpoint.

    Args:
        sentences (List[str]): A list of sentences for which embeddings need to be generated.
        checkpoint (Checkpoint): The checkpoint object used for generating embeddings.
        batch_size (int): The batch size to use during inference.
        batch_size2 (int): The size of the sub-batches to split the input sentences into.

    Returns:
        np.ndarray: An array of embeddings for the input sentences. 
        Shape of the array is (len(sentences), 32, 768) for bert-base-multilingual-cased or (len(sentences), 32, 128) for colbertv2.0
    """
    embeddings_list = []
    
    for i in range(0, len(sentences), batch_size2):
        # print(f"batch: {min(i+batch_size2, len(sentences))}/{len(sentences)}")

        batch_sentences = sentences[i:i+batch_size2]
        embeddings = get_query_emb(batch_sentences, checkpoint, batch_size)
        embeddings_list.append(embeddings)

        torch.cuda.empty_cache()
    
    combined_embeddings = np.concatenate(embeddings_list, axis=0)
    return combined_embeddings

def batch_generator_from_df(df, batch_size):
    num_samples = len(df)
    num_batches = num_samples // batch_size
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        offer_batch = df['name'].iloc[start_idx:end_idx].values
        true_match_batch = df['true_match'].iloc[start_idx:end_idx].values
        false_match_batch = df['false_match'].iloc[start_idx:end_idx].values
        
        yield list(offer_batch), list(true_match_batch), list(false_match_batch)

def prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint):

    offer_embs = get_query_emb_batch(offer_batch, checkpoint, batch_size=1000, batch_size2=10000)
    true_match_embs = get_query_emb_batch(true_match_batch, checkpoint, batch_size=1000, batch_size2=10000)
    false_match_embs = get_query_emb_batch(false_match_batch, checkpoint, batch_size=1000, batch_size2=10000)

    y_true = np.ones(len(true_match_embs))  # Метка 1 для пар (offer_embs[i], true_match_embs[i])
    y_false = np.zeros(len(false_match_embs))  # Метка 0 для пар (offer_embs[i], false_match_embs[i])
    
    # Создаем обучающий набор данных
    X_true = np.concatenate([offer_embs, true_match_embs], axis=1)
    X_false = np.concatenate([offer_embs, false_match_embs], axis=1)

    del offer_embs, true_match_embs, false_match_embs
    torch.cuda.empty_cache()

    X = np.concatenate([X_true, X_false])
    y = np.concatenate([y_true, y_false])

    # Перемешиваем данные
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = X[indices]
    y = y[indices]
    return X.reshape((X.shape[0], -1)), y

def XGB_test(bst, dtest, y_test):
    preds = bst.predict(dtest)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    Precision = precision_score(y_test, best_preds, average='macro')
    Recall = recall_score(y_test, best_preds, average='macro')
    Accuracy = accuracy_score(y_test, best_preds)
    return Precision, Recall, Accuracy

In [5]:

df_offers_shuffled = df_offers.sample(frac=1, random_state=42)

# Определяем размер тестовой выборки (например, 20%)
test_size = int(0.15 * len(df_offers_shuffled))

# Разделяем данные на тренировочную и тестовую выборки
df_train = df_offers_shuffled.iloc[:-test_size]
df_test = df_offers_shuffled.iloc[-test_size:]

# Пример: вывод размеров тренировочной и тестовой выборок
print("Размер тренировочной выборки:", len(df_train))
print("Размер тестовой выборки:", len(df_test))


Размер тренировочной выборки: 36312
Размер тестовой выборки: 6408


In [26]:
def train_xgboost_on_gpu(train_df, test_df, checkpoint, batch_size=32, batch_size2=128, num_round=100):
    # Гиперпараметры XGBoost
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'tree_method': 'gpu_hist'  # Использование GPU для обучения
    }

    # Генерация данных для обучения и тестирования
    train_generator = batch_generator_from_df(train_df, batch_size)
    test_generator = batch_generator_from_df(test_df, batch_size)

    # Инициализация модели XGBoost
    bst = None

    # Обучение модели XGBoost
    for i, (offer_batch, true_match_batch, false_match_batch) in enumerate(train_generator):
        print(f"Training batch {i+1}")

        X_train, y_train = prepare_X_y(offer_batch, true_match_batch, false_match_batch)

        dtrain = xgb.DMatrix(X_train, label=y_train)

        if bst is None:
            bst = xgb.train(params, dtrain, num_boost_round=num_round)
        else:
            bst = xgb.train(params, dtrain, num_boost_round=num_round, xgb_model=bst)

        del X_train, y_train, dtrain
        torch.cuda.empty_cache()

    # Подготовка тестового набора данных
    X_test, y_test = prepare_X_y(*next(test_generator))

    dtest = xgb.DMatrix(X_test, label=y_test)

    # Оценка качества модели на тестовом наборе данных
    Precision, Recall, Accuracy = XGB_test(bst, dtest, y_test)

    print(f"Precision: {Precision}, Recall: {Recall}, Accuracy: {Accuracy}")

    return bst

def train_xgboost1(train_df, checkpoint, batch_size=32):
    model = xgb.XGBClassifier(n_jobs=-1)

    train_generator = batch_generator_from_df(train_df, batch_size)
    
    for i, (offer_batch, true_match_batch, false_match_batch) in enumerate(train_generator):
        print(f"Training_batch: {min((i+1)*batch_size, len(train_df))}/{len(train_df)}")

        X_train, y_train = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)

        model.fit(X_train, y_train)
        torch.cuda.empty_cache()

    return model

"""
objective function: `biary:logistic`
Objective candidate: survival:aft
Objective candidate: binary:hinge
Objective candidate: multi:softmax
Objective candidate: multi:softprob
Objective candidate: rank:pairwise
Objective candidate: rank:ndcg
Objective candidate: rank:map
Objective candidate: reg:squarederror
Objective candidate: reg:squaredlogerror
Objective candidate: reg:logistic
Objective candidate: binary:logistic
Objective candidate: binary:logitraw
Objective candidate: reg:linear
Objective candidate: reg:pseudohubererror
Objective candidate: count:poisson
Objective candidate: survival:cox
Objective candidate: reg:gamma
Objective candidate: reg:tweedie
Objective candidate: reg:absoluteerror
"""
def train_xgboost(df, checkpoint, batch_size=32):
    params = {
    'eta': 0.3, 
    'max_depth': 200,  
    'objective': 'binary:logistic',  
    'num_class': 2,
    'gpu_id' : 0,
    'tree_method' : 'gpu_hist'
    }  

    # model = xgb.XGBClassifier(params)
    model = xgb.XGBClassifier(n_jobs=-1, tree_method='gpu_hist', predictor='gpu_predictor', objective = 'multi:softprob', num_class=2)

    offer_batch = list(df['name'])
    true_match_batch = list(df['true_match'])
    false_match_batch = list(df['false_match'])
    X_train, y_train = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)
    torch.cuda.empty_cache()
    
    for i in range(0, len(X_train), batch_size):
        print(f"batch: {min(i+batch_size, len(X_train))}/{len(X_train)}")

        X_train_batch = X_train[i:i+batch_size]
        y_train_batch = y_train[i:i+batch_size]

        torch.cuda.empty_cache()

        model.fit(X_train_batch, y_train_batch)

    return model

def test_xgboos1t(test_df, checkpoint, batch_size=32):
    test_generator = batch_generator_from_df(test_df, batch_size)
    # Подготовка тестового набора данных
    X_test, y_test = prepare_X_y(*next(test_generator), checkpoint)

    # Make predictions on the test set
    predictions = model.predict(X_test)

    # Evaluate the model performance
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: {:.2f}%".format(accuracy * 100))

# bst = train_xgboost_on_gpu(df_train, df_test, checkpoint, batch_size=50, batch_size2=50, num_round=100)

model = train_xgboost(df_train[:1000], checkpoint, batch_size=10)


batch: 10/2000
batch: 20/2000
batch: 30/2000
batch: 40/2000
batch: 50/2000
batch: 60/2000
batch: 70/2000
batch: 80/2000
batch: 90/2000
batch: 100/2000
batch: 110/2000
batch: 120/2000
batch: 130/2000
batch: 140/2000
batch: 150/2000
batch: 160/2000
batch: 170/2000
batch: 180/2000
batch: 190/2000
batch: 200/2000
batch: 210/2000
batch: 220/2000
batch: 230/2000
batch: 240/2000
batch: 250/2000
batch: 260/2000
batch: 270/2000
batch: 280/2000
batch: 290/2000
batch: 300/2000
batch: 310/2000
batch: 320/2000
batch: 330/2000
batch: 340/2000
batch: 350/2000
batch: 360/2000
batch: 370/2000
batch: 380/2000
batch: 390/2000
batch: 400/2000
batch: 410/2000
batch: 420/2000
batch: 430/2000
batch: 440/2000
batch: 450/2000
batch: 460/2000
batch: 470/2000
batch: 480/2000
batch: 490/2000
batch: 500/2000
batch: 510/2000
batch: 520/2000
batch: 530/2000
batch: 540/2000
batch: 550/2000
batch: 560/2000
batch: 570/2000
batch: 580/2000
batch: 590/2000
batch: 600/2000
batch: 610/2000
batch: 620/2000
batch: 630/2000
b

In [28]:
def test_xgboost(df, checkpoint, batch_size=32):
    offer_batch = list(df['name'])
    true_match_batch = list(df['true_match'])
    false_match_batch = list(df['false_match'])
    X_train, y_train = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)

    # Make predictions on the test set
    predictions = model.predict(X_train)

    # Evaluate the model performance
    accuracy = accuracy_score(y_train, predictions)
    print("Accuracy: {:.2f}%".format(accuracy * 100))

def test_data(df, checkpoint, batch_size=32):
    offer_batch = list(df['name'])
    true_match_batch = list(df['true_match'])
    false_match_batch = list(df['false_match'])
    X_train, y_train = prepare_X_y(offer_batch, true_match_batch, false_match_batch, checkpoint)

    return X_train, y_train

test_xgboost(df_test, checkpoint, batch_size=50)

# X_test, y_test = test_data(df_test, checkpoint, batch_size=50)

In [30]:
predictions = model.predict(X_test)

In [32]:
y_test

array([0., 1., 1., ..., 0., 1., 1.])

In [31]:
predictions

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [33]:
best_preds = np.asarray([np.argmax(line) for line in predictions])

In [34]:
best_preds

array([0, 1, 1, ..., 0, 0, 0])

In [35]:
accuracy = accuracy_score(y_test, best_preds)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 49.02%


In [8]:
model

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model                                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [9]:
def XGB_test(bst, dtest, y_test):
    preds = bst.predict(dtest)
    best_preds = np.asarray([np.argmax(line) for line in preds])

    Precision = precision_score(y_test, best_preds, average='macro')
    Recall = recall_score(y_test, best_preds, average='macro')
    Accuracy = accuracy_score(y_test, best_preds)
    return Precision, Recall, Accuracy

test_generator = batch_generator_from_df(df_test, 50)


In [10]:

# Подготовка тестового набора данных
X_test, y_test = prepare_X_y(*next(test_generator))


In [12]:
np.shape(X_test)

(100, 49152)

In [13]:
np.shape(y_test)

(100,)